In [ ]:
!pip install sk-video

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import skvideo
import skvideo.io

video = skvideo.io.vread("monte_carlo_low.mp4", as_grey=True)
video = np.moveaxis(video.reshape(video.shape[:-1]), 0, -1)

dt = 1 / 60 # 60 fps

In [ ]:
print(video.shape)

plt.imshow(video[:,:,5], cmap="gray")
plt.show()

In [ ]:
X = video.reshape(video.shape[0] * video.shape[1], -1)
X1 = X[:, 0:-2]
X2 = X[:, 1:-1]
U, Sigma, V = np.linalg.svd(X1, full_matrices=False)


In [ ]:
plt.plot(Sigma[:20])
plt.xlabel("Singular Values")
plt.title("First 20 Singular Values for Ski Drop Clip")
plt.show()

In [ ]:
# mc: rank = 1
#ski drop: rank = 10
rank = 10
S = U[:, 0:rank].T @ X2 @ V[:, 0:rank] @ np.diag(1. / Sigma[0:rank])
eigenvalues, eigenvectors = np.linalg.eig(S)
omega = np.log(eigenvalues) / dt
phi = U[:, 0:rank] @ eigenvectors

In [ ]:
y0, _, _, _ = np.linalg.lstsq(phi, X1[:, 0], rcond=None)

In [ ]:
ind = np.argmin(np.abs(omega))
X_low = y0[ind] * phi[:, ind] * np.exp(omega[ind] * (ind * dt))
X_sparse = X - np.abs(X_low)[:, None]


## Maybe move these? idk
X_low_norm = X_low / np.amax(np.abs(X_low))
X_sparse_norm = (X / np.abs(np.amax(np.abs(X), axis=0))) - np.abs(X_low_norm[:, None])


R = X_sparse_norm * (X_sparse_norm < 0)
X_low_imp = R[:, 0] + np.abs(X_low_norm)
X_sparse_imp = X_sparse_norm - R

In [ ]:
plt.imshow(X[:, 10].reshape(video.shape[0], video.shape[1]), cmap="gray")
plt.show()
plt.imshow((X_sparse_imp[:, 10]).reshape(video.shape[0], video.shape[1]), cmap="gray")
plt.show()

In [ ]:
# Check ranges of data
print(np.amin(np.abs(X_low_norm)))
print(np.amax(np.abs(X_low_norm)))
plt.imshow(np.abs(X_low_norm).reshape(video.shape[0], video.shape[1]), cmap="gray")
plt.show()

test_img = X[:, 0] / np.amax(X[:, 0])
print(np.amin(np.abs(test_img)))
print(np.amax(np.abs(test_img)))
plt.imshow(test_img.reshape(video.shape[0], video.shape[1]), cmap="gray")
plt.show()

fg = test_img - X_low_norm
print(np.amin(fg))
print(np.amax(fg))
plt.imshow(np.abs(fg).reshape(video.shape[0], video.shape[1]), cmap="gray")
plt.colorbar()
plt.show()